### Initial Observations

Due to the complex nature of computer security, CybORG's raw observations contain a lot of information presented in a standardised format which takes the form of a series of nested dictionaries and lists. It is recommended you use prettyprint whenever printing a CybORG observation.

We will begin by instantiating CybORG and looking at Red's initial observation.

In [ ]:
import random
import inspect
from os.path import dirname
from pprint import pprint

from CybORG import CybORG
from CybORG.Simulator.Scenarios import FileReaderScenarioGenerator

path = inspect.getfile(CybORG)
path = dirname(path) + f'/Simulator/Scenarios/scenario_files/Scenario1b.yaml'
sg = FileReaderScenarioGenerator(path)
cyborg = CybORG(scenario_generator=sg)

env = CybORG(sg)

results = env.reset(agent='Blue')
obs = results.observation
pprint(obs)

The dictionary above has two keys: 'success' and 'User0'. The success value indicates whether the previous action ran successfully, or whether it encountered an error. Since this is the start of the scenario, the success value is set to UNKNOWN.

The key 'User0' is a hostid, indicating its corresponding value is data about that host. Here the hostid is equal to the name of the host, altough hostids can also be ip_addresses depending on the previous action.

We will now take a closer look  examine the 'User0' dictionary.

In [ ]:
pprint(obs['User0'])

The 'User0' dictionary contains information about the host 'User0'. 'Interfaces' gives us networking information such as the host's ip address, which is randomized each scenario. 'Processes' lets us know any security-relevant processes running on the host that Red knows about; in this case Red begins with a SYSTEM level shell on User0. 'Sessions' lets us know any shells Red is aware of. Again, we can see it only sees its own starting shell. Finally, 'System info' tells us information about the operating system. For example, it is running Windows Server 2008 and has hostname 'User0'.

You may notice that 'Interface', 'Processes' and 'Sessions' all have lists as values. This is because a host can and usually will have multiple of these running at the same time.

We can now look at Blue's initial observation by calling the get_observation() method. Blue has access to the entire network so its initial observation is huge thus we will initially only print out the keys.

In [ ]:
blue_obs = env.get_observation('Blue')

print(blue_obs)

Printing out Blue's observation for 'User0', we can see it has has a different picture than Red. It has visibility of its own shells, but cannot detect red's.

In [ ]:
pprint(blue_obs['User0'])

### Red and Blue Observations

We will now import a rules-based red agent and run it one step to get a new observation.

In [ ]:
from CybORG.Agents import B_lineAgent

action_space = results.action_space
red_obs = results.observation
agent = B_lineAgent()

def step_red(obs, verbose=True):
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Red')
    obs = results.observation
    
    if verbose:
        print('Red Action:',action)
        print(76*'-')
        pprint(obs)
    
    return results

results = step_red(red_obs)
red_obs = results.observation

We can see that the agent has performed a 'DiscoverRemoteSystems' action. This represents a network scan discovering active ip addresses on a target subnet. Note these are now the hostids.

Note that the observation only contains information relevant to the previous action. We no longer have information given by the initial observation. This is because observations are designed to represent the output of real tools used by cybersecurity professionals, which only give extremely limited information at any one time. See the Wrapper tutorial for ways of gluing observations together to form a coherent picture.

Meanwhile because we haven't specified a Blue agent, blue team has been passively monitoring the network. However, it doesn't have any tools to detect Red's subnet scan, so its observation is virtually empty.

In [ ]:
print(env.get_observation('Blue'))

However, Red's next action is a port scan, which blue is able to detect.

In [ ]:
results = step_red(red_obs)
red_obs = results.observation

blue_obs = env.get_observation('Blue')
print(76*'-')
print('Blue Observation:')
print(76*'.')
pprint(blue_obs)
print(76*'.')



Both agents' observations are similar: a flurry of connections to a single ip address as the host is scanned for open ports.

The next action is an exploit establishing a shell for Red on 'User1'. Again both teams have similar observations, observing the connections generated by this.

In [ ]:
results = step_red(red_obs)
red_obs = results.observation

blue_obs = env.get_observation('Blue')
print(76*'-')
print('Blue Observation:')
print(76*'.')
pprint(blue_obs)

Note that there is a 5% chance that Red's exploit fails to register to be detected by Blue.

The final action in this demo sees Red execute a privilege escalation. Red gets a SYSTEM shell, but Blue is unable to see this activity whatsoever.

In [ ]:
results = step_red(red_obs)
red_obs = results.observation

blue_obs = env.get_observation('Blue')
print(76*'-')
print('Blue Observation:')
print(76*'.')
pprint(blue_obs)

Fortunately for Blue, Red's Privilege Escalation leaves malware on the system which can be detected using the Analyse action.

In [ ]:
from CybORG.Simulator.Actions import Analyse

host = env.get_last_action('Red').hostname
action = Analyse(session=0,agent='Blue',hostname=host)

results = env.step(action=action,agent='Blue')

pprint(results.observation)

Here we see 'User1' has a 'Files' key. The correspending dictionary has been passed through a tool and the 'Density' parameter indicates a high probability this is malware.